<a href="https://colab.research.google.com/github/migdashn/Deep-Learning-Projects/blob/main/Colorozation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from sklearn.utils import shuffle
from torchvision import datasets, transforms
from glob import glob

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

In [ ]:

cifar_data = datasets.CIFAR10('data', train=True, download=True, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(cifar_data, batch_size=64, shuffle=True)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data


In [ ]:
#TODO checks to the data to see how to use it

To convert a colorized Tensor to black and white, you can use the grayscale method, which takes a 3-channel image and converts it to a single-channel image by combining the values of the red, green, and blue channels using a weighted average. Here's an example of how you can convert a color image represented as a Tensor to a grayscale image using PyTorch:

In [ ]:
# Create a color image Tensor
color_tensor = torch.rand(3, 256, 256)

# Convert the Tensor to grayscale using the grayscale method
gray_tensor = torch.mean(color_tensor, dim=0, keepdim=True)

Alternatively, you can use the following formula to convert RGB image to grayscale image where Y = 0.299 * R + 0.587 * G + 0.114 * B

In [ ]:
gray_tensor = 0.299 * color_tensor[0,:,:] + 0.587 * color_tensor[1,:,:] + 0.114 * color_tensor[2,:,:]

In [ ]:
#TODO create a black and white data using one of the methods

Generator

In [ ]:
class Encoder(nn.Module):
    def __init__(self, in_channels, num_hiddens, latent):
        super(Encoder, self).__init__()
        self.conv = nn.Conv2d(input, input//2, kernel_size=(4,4), stride=(2,2))
        self.norm = nn.BatchNorm2d(input//2)
        self.lr = nn.LeakyReLU(0.2)

    def forward(self,inputs,ifNorm):
        x = self.conv(inputs)
        if ifNorm == True:
          x = self.norm(x)
        x = self.lr(x)
        return x




In [ ]:
class Decoder(nn.Module):
    def __init__(self, input, ifDrop):
        super(Decoder, self).__init__()
        self.conv = nn.ConvTranspose2d(input, input*2, kernel_size=(4,4), stride=(2,2))
        self.norm = nn.BatchNorm2d(input*2)
        self.drop = nn.Dropout2d(0.5)
        self.rel = nn.ReLU()

    def forward(self,inputs,ifDrop):
        x = self.conv(inputs)
        x = self.norm(x)
        if ifDrop == True:
          x = self.drop(x)
        x = self.rel(x)
        return x

In [ ]:
class Generator(nn.Module):
    def __init__(self, in_channels):
        super(Generator, self).__init__()
        self.encode = Encoder()
    def forward(self, x):

      return x



In [ ]:
/* u-net gan model using for pix2pix in pytorch */

import torch
import torch.nn as nn
import torch.nn.functional as F
class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        x = self.conv(x)
        return x
class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)
    def forward(self, x):
        x = self.conv(x)
        return x
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )
    def forward(self, x):
        x = self.mpconv(x)
        return x
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()
        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)
        self.conv = double_conv(in_ch, out_ch)
    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, (diffX // 2, diffX - diffX//2,
                        diffY // 2, diffY - diffY//2))
        # for padding issues, see 
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x
class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)
    def forward(self, x):
        x = self.conv(x)
        return x
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.inc = inconv(n_channels, 64)
        self.

In [ ]:
def Generator():
    inputs = Input(shape=(256,256,3), name="InputLayer")
    down_stack = [downsample(64, apply_batchnorm=False),
        downsample(128),
        downsample(256),
        downsample(512),
        downsample(512),
        downsample(512),
        downsample(512),]
    encoding = downsample(512)

    up_stack = [upsample(512, apply_dropout=True),
        upsample(512, apply_dropout=True),
        upsample(512, apply_dropout=True),
        upsample(512),
        upsample(256),
        upsample(128),
        upsample(64),]

    x = inputs 
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
    
    x = encoding(x)

    skips = reversed(skips)
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = concatenate([x, skip])
    
    init = RandomNormal(stddev=0.02)
    out = Conv2DTranspose(3, kernel_size=4, strides=2, kernel_initializer=init, activation='tanh', padding='same')

    out = out(x)

    gen = Model(inputs=inputs, outputs=out, name="Generator")
    return gen


Encoder Block

In [ ]:
def downsample(filters, apply_batchnorm=True):

    model = Sequential(
        Conv2D(filters, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal', use_bias=False),
    )

    if apply_batchnorm:
        model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    return model

Decoder BLock

In [ ]:
def upsample(filters, apply_dropout=False):

    model = Sequential()
    model.add(Conv2DTranspose(
        filters,
        kernel_size=4,
        strides=2,
        padding='same',
        kernel_initializer='he_normal',
        use_bias=False))
    model.add(BatchNormalization())
    
    if apply_dropout:
        model.add(Dropout(0.5))
    
    model.add(ReLU())

    return model

Discriminator Model

In [ ]:
def Discriminator():
    init = RandomNormal(stddev=0.02)

    image = Input(shape=(256,256,3), name="ImageInput")
    target = Input(shape=(256,256,3), name="TargetInput")
    x = concatenate([image, target])

    x = downsample(64, apply_batchnorm=False)(x)
    x = downsample(128)(x)
    x = downsample(512)(x)

    x = ZeroPadding2D()(x)
    x = Conv2D(512, kernel_size=4, strides=1, kernel_initializer=init, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = ZeroPadding2D()(x)

    x = Conv2D(1, kernel_size=4, kernel_initializer=init)(x)

    model = Model(
        inputs=[image, target],
        outputs=x,
        name="Discriminator"
    )

    return model